In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import interact, Dropdown, IntSlider

%matplotlib notebook
plt.style.use('grayscale')

In [2]:
! tree -d ..

..
├── data
│   ├── train
│   │   ├── axial
│   │   ├── coronal
│   │   └── sagittal
│   └── valid
│       ├── axial
│       ├── coronal
│       └── sagittal
├── exp
└── mrnet-fastai

11 directories


In [3]:
! ls ../data/train/axial | head -n 5

0000.npy
0001.npy
0002.npy
0003.npy
0004.npy
ls: write error: Broken pipe


In [4]:
data_path = Path('../data')
train_path = data_path/'train'
valid_path = data_path/'valid'

In [5]:
train_abnl = pd.read_csv(data_path/'train-abnormal.csv', header=None,
                       names=['Case', 'Abnormal'], 
                       dtype={'Case': str, 'Abnormal': np.int64})
print(train_abnl.groupby('Abnormal').count())
train_abnl.head()

          Case
Abnormal      
0          217
1          913


,Case,Abnormal
0,0000,1
1,0001,1
2,0002,1
3,0003,1
4,0004,1


In [6]:
train_acl = pd.read_csv(data_path/'train-acl.csv', header=None,
                       names=['Case', 'ACL_tear'], 
                       dtype={'Case': str, 'ACL_tear': np.int64})
print(train_acl.groupby('ACL_tear').count())
train_acl.head()

          Case
ACL_tear      
0          922
1          208


,Case,ACL_tear
0,0000,0
1,0001,1
2,0002,0
3,0003,0
4,0004,0


In [7]:
train_meniscus = pd.read_csv(data_path/'train-meniscus.csv', header=None,
                       names=['Case', 'Meniscus_tear'], 
                       dtype={'Case': str, 'Meniscus_tear': np.int64})
print(train_meniscus.groupby('Meniscus_tear').count())
train_meniscus.head()

               Case
Meniscus_tear      
0               733
1               397


,Case,Meniscus_tear
0,0000,0
1,0001,1
2,0002,0
3,0003,1
4,0004,0


In [8]:
def load_one_stack(case, data_path=train_path, plane='coronal'):
    fpath = data_path/plane/'{}.npy'.format(case)
    return np.load(fpath)

def load_stacks(case, data_path=train_path):
    x = {}
    planes = ['coronal', 'sagittal', 'axial']
    for i, plane in enumerate(planes):
        x[plane] = load_one_stack(case, data_path, plane=plane)
    return x

def load_partial_stacks(case, data_path=train_path, slice_limit=None):
    x = {}
    planes = ['coronal', 'sagittal', 'axial']
    if not slice_limit:
        return load_stacks(case, data_path)
    else:
        for i, plane in enumerate(planes):
            data = load_one_stack(case, data_path, plane)
            if slice_limit >= data.shape[0]:
                x[plane] = data
            else:
                mid_slice = data.shape[0] // 2
                lower = mid_slice - (slice_limit // 2)
                upper = mid_slice + (slice_limit // 2)
                x[plane] = data[lower:upper, :, :]
    return x
    

In [9]:
case = train_abnl.Case[0]
x = load_one_stack(case)
print(x.shape)
print(x.max())

(36, 256, 256)
255


In [10]:
x = load_stacks(case)
x.keys()

dict_keys(['coronal', 'sagittal', 'axial'])

In [11]:
class KneePlot():
    def __init__(self, x: dict, figsize=(10, 10)):
        self.x = x
        self.planes = list(x.keys())
        self.slice_nums = {plane: self.x[plane].shape[0] for plane in self.planes}
        self.figsize = figsize
    
    def _plot_slices(self, plane, im_slice): 
        fig, ax = plt.subplots(1, 1, figsize=self.figsize)
        ax.imshow(self.x[plane][im_slice, :, :])
        plt.show()
    
    def draw(self):
        planes_widget = Dropdown(options=self.planes)
        plane_init = self.planes[0]
        slice_init = self.slice_nums[plane_init] - 1
        slices_widget = IntSlider(min=0, max=slice_init, value=slice_init//2)
        def update_slices_widget(*args):
            slices_widget.max = self.slice_nums[planes_widget.value] - 1
            slices_widget.value = slices_widget.max // 2
        planes_widget.observe(update_slices_widget, 'value')
        interact(self._plot_slices, plane=planes_widget, im_slice=slices_widget)
    
    def resize(self, figsize): self.figsize = figsize


In [12]:
plot = KneePlot(x)
plot.draw()

interactive(children=(Dropdown(description='plane', options=('coronal', 'sagittal', 'axial'), value='coronal')…